In [36]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

In [74]:
trans = [{"Date":"2021/01/01","Symbol":"BTU","Type":"BUY","entry_price":45.50,"units":20},
         {"Date":"2021/02/01","Symbol":"BTU","Type":"BUY","entry_price":50.25,"units":80},
         {"Date":"2021/02/02","Symbol":"BTU","Type":"SELL","entry_price":32.25,"units":15},
         {"Date":"2022/02/01","Symbol":"AMC","Type":"BUY","entry_price":10.13,"units":50},
         {"Date":"2022/05/01","Symbol":"BTU","Type":"BUY","entry_price":8.10,"units":45},
         {"Date":"2021/03/01","Symbol":"BTU","Type":"BUY","entry_price":8.00,"units":150},
         {"Date":"2021/02/01","Symbol":"F","Type":"BUY","entry_price":14.13,"units":50},
         {"Date":"2021/02/01","Symbol":"F","Type":"BUY","entry_price":10.1,"units":30},
         {"Date":"2021/02/01","Symbol":"F","Type":"BUY","entry_price":3.55,"units":150},
         {"Date":"2023/02/01","Symbol":"ADBE","Type":"BUY","entry_price":10.55,"units":150},
         {"Date":"2023/05/01","Symbol":"ADBE","Type":"SELL","entry_price":11.55,"units":150},
         {"Date":"2021/02/01","Symbol":"F","Type":"SELL","entry_price":15.55,"units":50}]


In [75]:

stock_list = ['BAC', 'C', 'HIG', 'LEN', 'MS'] #Input
start_date = '2019-01-01' #Input
end_date = '2021-12-31'#Input

In [198]:
class PortfolioStats:
    def __init__(self, name, init_balance, currency, list_of_transactions, start_date, end_date) -> None:
        self.name = name 
        self.init_balance = init_balance
        self.currency = currency
        self.transactions = list_of_transactions
        self.start_date = start_date
        self.end_date = end_date
        
    def get_trans_as_df(self):
        trans = pd.DataFrame.from_dict(self.transactions)
        trans["Date"] = pd.to_datetime(trans["Date"])
        trans["units"] = np.where( trans["Type"]=="SELL",-trans["units"],trans["units"]  )
        #trans["cost_basis"] = trans["units"] * trans["entry_price"]
        trans.set_index("Date")
        return trans
    
    def get_ptf_positions(self):
        transactions = self.get_trans_as_df()
        trans_data = transactions[["Symbol","units"]]
        tdf = trans_data.groupby("Symbol").sum()
        non_zero_pos = tdf["units"] > 0
        tdf = tdf[non_zero_pos]
        ptf_pos =  tdf.to_dict()
        return ptf_pos
    
    def get_open_positions(self):
        transactions = self.get_trans_as_df()
        trans_data = transactions[["Symbol","units"]]
        tdf = trans_data.groupby("Symbol").sum()
        non_zero_pos = tdf["units"] > 0
        tdf = tdf[non_zero_pos]
        open_pos =  tdf.to_dict()
        
        open_positions = []
        for position in open_pos["units"]:
            
            buy_transactions = {}
            for transaction in trans:
                if transaction["Type"] == "BUY" and transaction["Symbol"] == position:
                    symbol = transaction["Symbol"]
                    price = transaction["entry_price"]
                    units = transaction["units"]
                    if symbol in buy_transactions:
                        buy_transactions[symbol]["total_cost"] += price * units
                        buy_transactions[symbol]["total_units"] += units
                    else:
                        buy_transactions[symbol] = {"total_cost": price * units, "total_units": units}
            
            
            for symbol, data in buy_transactions.items():
                portfolio_pos = {}
                avg_price = data["total_cost"] / data["total_units"]
                data["avg_price"] = avg_price
                portfolio_pos[symbol] = data
                open_positions.append(portfolio_pos)
                print(f"Symbol: {symbol}, Average Buy Price: {avg_price:.2f}")
        return open_positions
    
    def get_open_position_prices(self):
        stock_list = list(self.get_ptf_positions()["units"].keys())
        # Stažení dat z Yahoo Finance
        data = yf.download(stock_list , period="1d")["Adj Close"]
        return data
        
        


In [199]:
def get_data(stock, start, end):
    try:
        stock_data = yf.download(stock_list, start=start_date, end=end_date)["Adj Close"]
        print(stock_data.head())
        return stock_data
    except Exception as e:
        print(f'No data found for {stock}.{e}')

In [200]:
get_data("AMC",start_date,end_date)

[*********************100%***********************]  5 of 5 completed
                  BAC          C        HIG        LEN         MS
Date                                                             
2019-01-02  22.641018  45.715565  39.206615  38.050537  35.365761
2019-01-03  22.278183  44.887173  38.664833  37.660137  34.735489
2019-01-04  23.203415  47.081993  39.856750  39.240810  36.153610
2019-01-07  23.185276  47.491924  39.712273  40.354900  36.512524
2019-01-08  23.139921  47.363819  39.179531  40.840530  36.284931


,BAC,C,HIG,LEN,MS
Date,,,,,
2019-01-02,22.641018,45.715565,39.206615,38.050537,35.365761
2019-01-03,22.278183,44.887173,38.664833,37.660137,34.735489
2019-01-04,23.203415,47.081993,39.856750,39.240810,36.153610
2019-01-07,23.185276,47.491924,39.712273,40.354900,36.512524
2019-01-08,23.139921,47.363819,39.179531,40.840530,36.284931
...,...,...,...,...,...
2021-12-23,43.129517,56.696262,66.396019,109.225639,94.377960
2021-12-27,43.343128,57.110584,66.950363,110.609848,95.385025
2021-12-28,43.401390,57.082336,67.242149,111.613892,94.976494


In [201]:
p1 = PortfolioStats("MyTestPortfolio",10000,"USD",trans, start_date, end_date)

In [191]:
data["BAC"][-1]

27.09000015258789

In [204]:
#pos = p1.get_open_positions()
ps2 = p1.get_open_positions()

Symbol: AMC, Average Buy Price: 10.13
Symbol: BTU, Average Buy Price: 22.02
Symbol: F, Average Buy Price: 6.70


In [205]:
ps2

[{'AMC': {'total_cost': 506.50000000000006,
   'total_units': 50,
   'avg_price': 10.13}},
 {'BTU': {'total_cost': 6494.5,
   'total_units': 295,
   'avg_price': 22.015254237288136}},
 {'F': {'total_cost': 1542.0,
   'total_units': 230,
   'avg_price': 6.7043478260869565}}]

SPY: No data found for this date range, symbol may be delisted


Series([], Name: Close, dtype: float64)

In [139]:
a

['F']

In [94]:
pos[i][symbol]["total_cost"]
pos[0]["AMC"]["total_cost"]

506.50000000000006

In [101]:

import yfinance as yf
yca = yf.Ticker("YCA.L").history(interval="1m", period = "1d")


404.79998779296875

In [206]:

data = yf.download("AAPL",'2015-1-1')

[*********************100%***********************]  1 of 1 completed


In [207]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,27.847500,27.860001,26.837500,27.332500,24.531763,212818400
2015-01-05,27.072500,27.162500,26.352501,26.562500,23.840666,257142000
2015-01-06,26.635000,26.857500,26.157499,26.565001,23.842915,263188400
2015-01-07,26.799999,27.049999,26.674999,26.937500,24.177244,160423600
2015-01-08,27.307501,28.037500,27.174999,27.972500,25.106184,237458000
...,...,...,...,...,...,...
2023-05-08,172.479996,173.850006,172.110001,173.500000,173.260345,55962800
2023-05-09,173.050003,173.539993,171.600006,171.770004,171.532745,45326900
2023-05-10,173.020004,174.029999,171.899994,173.559998,173.320267,53724500
